## Using Python to Query MySQL
This notebook demonstrates using a couple of different database connectivity libraries to connect to and query a MySQL database.
- **PyMySQL** library
- MySQL's Native **mysql.connector** library
- **SqlAlchemy** library

### 1.0. Prerequisites

#### 1.1. First, you must **install** the libaries into your *python* environment by executing the following commands in a *Terminal window*
- python -m pip install PyMySQL
- python -m pip install mysql.connector-python
- python -m pip install sqlalchemy

#### 1.2. Next, as with all Jupyter Notebooks, you need to **Import** the libaries that you'll be working with in the notebook,

In [ ]:
import os
import pymysql
import mysql.connector
from sqlalchemy import create_engine

import pandas as pd
import matplotlib.pyplot as plt

#### 1.3. And then, Assign Connection Variables for the MySQL Server & Database with which You'll be Working 

In [ ]:
host_name = "localhost" #"compid-mysql.mysql.database.azure.com"
host_ip = "127.0.0.1"
port = "3306"

user_id = "root"
pwd = "Passw0rd123"
db_name = "northwind"

### 2.0. Using the PyMySQL Library
#### 2.1. Using a Cursor to Iterate the Rows Returned

In [ ]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor()

try:
    cursor.execute('SELECT * FROM products;')
    
    for row in cursor.fetchmany(size=5):
        print(row)
        
    cursor.close()
    
except:
    print ("Error: unable to fetch data")
    
conn.close()

In [ ]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)
cursor = conn.cursor(pymysql.cursors.DictCursor)

try:
    cursor.execute('SELECT * FROM products ORDER BY list_price DESC LIMIT 5;')
    
    for row in cursor.fetchall():
        print(row)
        
    cursor.close()
    
except:
    print ("Error: unable to fetch data")
    
conn.close()

#### 2.2. Using the Pandas read_sql() Method to Return a DataFrame

In [ ]:
conn = pymysql.connect(host=host_name, user=user_id, password=pwd, database=db_name)

df = pd.read_sql("SELECT * FROM products ORDER BY list_price DESC;", conn)

conn.close()
df.head()

### 3.0. Using the MySQL Native Connection Library
#### 3.1. Using a Cursor to Iterate the Rows Returned

In [ ]:
lbound = 15.00
ubound = 20.00

sql_query = """
    SELECT id AS product_id
        , product_name
        , list_price
    FROM northwind.products
    WHERE list_price BETWEEN %s AND %s
    ORDER BY list_price DESC;
"""

In [ ]:
conn = mysql.connector.connect(user=user_id, password=pwd,
                               host=host_name, database=db_name,
                               auth_plugin='mysql_native_password')

In [ ]:
cursor = conn.cursor()

try:
    cursor.execute(sql_query, (lbound, ubound))
    
    for (product_id, product_name, list_price) in cursor:
        print("{}, {} was sold for {}".format(product_id, product_name, list_price))
        
    cursor.close()
    
except:
    print ("Error: unable to fetch data")   

In [ ]:
cursor = conn.cursor()

try:
    cursor.execute(sql_query, (lbound, ubound))
    
    row = cursor.fetchone() 
    while row:
        print(row)
        row = cursor.fetchone()
        
    cursor.close()
    
except:
    print ("Error: unable to fetch data")

In [ ]:
conn.close()

#### 3.2. Using Pandas read_sql() Method to Return a DataFrame

In [ ]:
lbound = 15.00
ubound = 20.00

sql_query = """
    SELECT id AS product_id
        , product_name
        , list_price
    FROM northwind.products
    WHERE list_price BETWEEN {0} AND {1}
    ORDER BY list_price DESC;
""".format(lbound, ubound)

print(sql_query)

In [ ]:
configs = {
    'user': user_id,
    'password': pwd,
    'host': host_name,
    'database': db_name,
    'auth_plugin': 'mysql_native_password',
    'raise_on_warnings': True
}

conn = mysql.connector.connect(**configs)

df = pd.read_sql(sql_query, conn)

conn.close()
df.tail()

### 4.0. Using the SQLAlchemy Connection Library

In [ ]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"

sqlEngine = create_engine(conn_str, pool_recycle=3600)
conn = sqlEngine.connect()

df = pd.read_sql(sql_query, conn);

conn.close()
df.head()

### 5.0 Define Helper Functions to Encapsulate and Abstract the Implementation Details

In [ ]:
sql_query = """
    SELECT id AS product_id
        , product_name
        , list_price
    FROM northwind.products
    ORDER BY list_price DESC;
"""

#### 5.1. Using Individual Connection Parameters

In [ ]:
def get_pymysql_dataframe(host, user, password, database_name, sql_query_string):
    connection = pymysql.connect(host=host, user=user, password=password, database=database_name)
    dframe = pd.read_sql(sql_query_string, connection)
    connection.close()
    
    return dframe

In [ ]:
df = get_pymysql_dataframe(host_name, user_id, pwd, db_name, sql_query)
df.head()

In [ ]:
print("Shape: {}\n".format(df.shape))

#### 5.1.1. Using SqlAlchemy

In [ ]:
def get_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe

In [ ]:
df = get_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, sql_query)
df.head(3)

In [ ]:
print(f"Shape: {df.shape[0]} Observations x {df.shape[1]} Features")

#### 5.2. Using a Configurations Dictionary

In [ ]:
def get_mysql_dataframe(sql_query_string, args):
    connection = mysql.connector.connect(**args)
    dframe = pd.read_sql(sql_query_string, connection)
    connection.close()
    
    return dframe

In [ ]:
dframe = get_mysql_dataframe(sql_query, configs)
dframe.tail(3)

In [ ]:
print(f"Shape: {dframe.shape[0]} Observations x {dframe.shape[1]} Features")

### 6.0. Writing a Pandas DataFrame to a SQL Database

In [ ]:
def insert_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, df, table_name):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    df.to_sql(table_name, con=connection, if_exists='replace') #, index_label='product_id');  'append'
    connection.close()

In [ ]:
insert_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, dframe, 'dim_products')

In [ ]:
df = get_sqlalchemy_dataframe(user_id, pwd, host_name, db_name, 'SELECT * FROM dim_products')
df.head()

### 7.0. Explore Pandas DataFrames' Capabilities
#### 7.1. Display the Data Type of Each Feature

In [ ]:
sql_query = "SELECT * FROM northwind.products;"

df = get_mysql_dataframe(sql_query, configs)

In [ ]:
df.dtypes

#### 7.1. Inspect the Cardinality (number of unique values) of each Feature

In [ ]:
df.nunique()

In [ ]:
unique_values = []

for column in df.columns:
    unique_values.append(df[column].unique())
    
data = list(zip(df.columns, unique_values))    
    
pd.DataFrame(data, columns=['Feature', 'Unique Values'])

#### 7.2. Display any Missing (NULL) values

In [ ]:
df.isnull().sum().sort_values(ascending=True)

#### 7.3. Separate Numerical and Categorical Features

In [ ]:
numerical_cols = [col for col in df.columns if df.dtypes[col] != 'O']
categorical_cols = [col for col in df.columns if col not in numerical_cols]

print(numerical_cols)
print(categorical_cols)

#### 7.4. Evaluate the Statistical Distribution of the Numerical Features

In [ ]:
df[numerical_cols].describe()

#### 7.5. Write the Contents of the DataFrame to a Comma-Separated Values (CSV) File

In [ ]:
data_dir = os.path.join(os.getcwd(), 'data')
dest_file = os.path.join(data_dir, 'northwind_products.csv')

df.to_csv(dest_file)